# Generate csv file containing DFT data for the BM-dataset

In this notebook the DFT BM-dataset is processed to generate a .csv file containing the information related to the DFT calculations that is used to test the GNN model predictions.

In [1]:
import os, sys
from subprocess import PIPE, Popen

import pandas as pd

In [2]:
BM_path = "/home/smorandi/Desktop/gnn_eads/data/BM_dataset"  # path to the BM dataset

In [3]:
Label, Family, Metal, Etot_DFT, Eslab_DFT, Emol_DFT = [], [], [], [], [], []
for family in os.listdir(BM_path):
    print(family)
    family_path = os.path.join(BM_path, family)
    for (dirpath, dirnames, filenames) in os.walk(family_path):
        for calc in dirnames:
            if "-" not in calc:
                continue
            if "0000" in calc:
                continue
            calc_path = os.path.join(dirpath, calc)
            print(calc_path)
            Label.append(calc)
            Family.append(family)
            Metal.append(calc.split("-")[0])
            p1 = Popen(["grep", "energy  w", os.path.join(calc_path, "OUTCAR")], stdout=PIPE)
            p2 = Popen(["tail", "-1"], stdin=p1.stdout, stdout=PIPE)
            p3 = Popen(["awk", "{print $NF}"], stdin=p2.stdout, stdout=PIPE)
            Etot_DFT.append(float(p3.communicate()[0]))
            gas_path = os.path.join(dirpath, calc.split("-")[-1])
            slab_path = os.path.join(dirpath, calc.split("-")[0] + "-" + "0000")
            p1 = Popen(["grep", "energy  w", os.path.join(gas_path, "OUTCAR")], stdout=PIPE)
            p2 = Popen(["tail", "-1"], stdin=p1.stdout, stdout=PIPE)
            p3 = Popen(["awk", "{print $NF}"], stdin=p2.stdout, stdout=PIPE)
            Emol_DFT.append(float(p3.communicate()[0]))
            p1 = Popen(["grep", "energy  w", os.path.join(slab_path, "OUTCAR")], stdout=PIPE)
            p2 = Popen(["tail", "-1"], stdin=p1.stdout, stdout=PIPE)
            p3 = Popen(["awk", "{print $NF}"], stdin=p2.stdout, stdout=PIPE)
            if family == "Plastics" or (family == "Polyurethanes" and calc.split("-")[0] == "au"):
                Eslab_DFT.append(float(p3.communicate()[0])*4)
            else:
                Eslab_DFT.append(float(p3.communicate()[0]))

df = pd.DataFrame({"Label": Label, "Family": Family, "Metal": Metal, "Etot_DFT": Etot_DFT, "Eslab_DFT": Eslab_DFT, "Emol_DFT": Emol_DFT})
df["Eads_DFT"] = df["Etot_DFT"] - df["Eslab_DFT"] - df["Emol_DFT"]
df.to_csv("/home/smorandi/Desktop/gnn_eads/data/BM_dataset/DFT_BM.csv", index=False)
            

Biomass
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ni-mol1
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ni-mol2
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ni-mol3
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ni-mol4
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ni-mol5
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ru-mol1
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ru-mol2
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ru-mol3
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ru-mol4
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Biomass/ru-mol5
Polyurethanes
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Polyurethanes/ag-mol16
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Polyurethanes/ag-mol17
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Polyurethanes/ag-mol18
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Polyurethanes/ag-mol19
/home/smorandi/Desktop/gnn_eads/data/BM_dataset/Polyur

In [4]:
df

,Label,Family,Metal,Etot_DFT,Eslab_DFT,Emol_DFT,Eads_DFT
0,ni-mol1,Biomass,ni,-699.003184,-429.518132,-266.620332,-2.864720
1,ni-mol2,Biomass,ni,-698.489544,-429.518132,-266.582128,-2.389283
2,ni-mol3,Biomass,ni,-585.906515,-429.518132,-153.601859,-2.786525
3,ni-mol4,Biomass,ni,-600.110308,-429.518132,-168.674337,-1.917839
4,ni-mol5,Biomass,ni,-593.315922,-429.518132,-162.199307,-1.598483
5,ru-mol1,Biomass,ru,-995.708889,-725.440080,-266.620332,-3.648478
6,ru-mol2,Biomass,ru,-995.174450,-725.440080,-266.582128,-3.152242
7,ru-mol3,Biomass,ru,-882.455468,-725.440080,-153.601859,-3.413530
8,ru-mol4,Biomass,ru,-896.613084,-725.440080,-168.674337,-2.498668
9,ru-mol5,Biomass,ru,-889.631857,-725.440080,-162.199307,-1.992470
